In [0]:
from __future__ import absolute_import, division, print_function

In [0]:
import tensorflow as tf
import numpy as np

In [4]:
print(tf.__version__)

1.13.1


In [0]:
# Recoleccion de la data
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [6]:
# Normalizacion de datos
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train.shape, x_test.shape

((60000, 28, 28), (10000, 28, 28))

In [7]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape, type(x_train), type(y_train)

((60000, 28, 28),
 (10000, 28, 28),
 (60000,),
 (10000,),
 numpy.ndarray,
 numpy.ndarray)

In [8]:
# Modelo: Keras creacion
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.summary()

estimator = tf.keras.estimator.model_to_estimator(keras_model=model)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provi

In [9]:
# Data ingest
def read_dataset(x, y, mode, batch_size=64):
  x_ds = tf.data.Dataset.from_tensor_slices(x)
  y_ds = tf.data.Dataset.from_tensor_slices(y[:,np.newaxis])
  
  ds = tf.data.Dataset.zip((x_ds, y_ds))
  
  if mode == tf.estimator.ModeKeys.TRAIN:
    num_epochs = 5
    ds = ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=batch_size*10))
  else:
    num_epochs = 1
  
  ds = ds.batch(batch_size).repeat(num_epochs).prefetch(tf.data.experimental.AUTOTUNE)
  return ds

def train_input_fn():
  return read_dataset(x = x_train,#<-------
                      y = y_train,#<-------
                      mode = tf.estimator.ModeKeys.TRAIN)

def eval_input_fn():
  return read_dataset(x = x_test,#<-------
                      y = y_test,#<-------
                      mode = tf.estimator.ModeKeys.EVAL)

# Produccion: data
train_spec = tf.estimator.TrainSpec(input_fn = train_input_fn,
                                   max_steps = 1000*5)
eval_spec = tf.estimator.EvalSpec(input_fn = eval_input_fn,
                                 steps = 500)

# Entrenamiento
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/tmp/tmpfc3cojxj/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ('/tmp/tmpfc3cojxj/keras/keras_model.ckpt',)
INFO:tensorflow:Warm-starting variable: dense/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_1/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dens

({'global_step': 5000,
  'loss': 0.06419535,
  'sparse_categorical_accuracy': 0.9811},
 [])